In [ ]:
# install Libraries 
!pip install qiskit-finance

Credit Risk Portfolio analysis
Portfolio analysis is the process of evaluating and assessing a collection of investments, known as a portfolio, to understand its performance, risks, and potential returns.
VaR is an acronym of ‘Value at Risk


In [2]:
# Import Libraries to be used for the Code
import numpy as np
import matplotlib.pyplot as plt

from qiskit import QuantumRegister, QuantumCircuit
from qiskit.circuit.library import IntegerComparator
from qiskit.algorithms import IterativeAmplitudeEstimation, EstimationProblem
from qiskit_aer.primitives import Sampler

In [ ]:
from qiskit_finance.circuit.library import GaussianConditionalIndependenceModel as GCI
# set problem parameters
n_z = 2
z_max = 2
z_values = np.linspace(-z_max, z_max, 2**n_z)
p_zeros = [0.15, 0.25]
rhos = [0.1, 0.05]
lgd = [1, 2]
K = len(p_zeros)
alpha = 0.05
u = GCI(n_z, z_max, p_zeros, rhos)

In [ ]:
u_measure = u.measure_all(inplace=False)
sampler = Sampler()
job = sampler.run(u_measure)
binary_probabilities = job.result().quasi_dists[0].binary_probabilities()

#https://qiskit.org/ecosystem/finance/tutorials/09_credit_risk_analysis.html
QAE - quantum ampitide estimation

https://medium.com/@samaybhagat/building-a-quantum-portfolio-optimizer-with-ibms-qiskit-c39f3a8810f4
https://qiskit.org/ecosystem/finance/tutorials/01_portfolio_optimization.html


Implementing MC simulations using Quantum algorithm (Financial domain)



In [ ]:
# initialization

import numpy as np
import matplotlib.pyplot as plt
import datetime 

# importing qiskit tools

from qiskit import Aer
from qiskit.algorithms import VQE, NumPyMinimumEigensolver
from qiskit.algorithms.optimizers import COBYLA
from qiskit.circuit.library import TwoLocal
from qiskit.utils import QuantumInstance
from qiskit_finance.applications.optimization import PortfolioOptimization
from qiskit_finance.data_providers import RandomDataProvider
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization.applications import OptimizationApplication
from qiskit_optimization.converters import QuadraticProgramToQubo

In [ ]:
# establishing the number of assets (same as # of cubits)

num_assets = 4
seed = 987

In [ ]:
# select randomly generated data through RandomDataProvider

stocks = [("TICKER%s" % i) for i in range(num_assets)]
data = RandomDataProvider(
    tickers=stocks,
    start=datetime.datetime(2021, 1, 1),
    end=datetime.datetime(2021, 1, 30),
    seed=seed,
)

In [ ]:
# calculate expected return and covariance from generated data

data.run()
ER = data.get_period_return_mean_vector()
cov = data.get_period_return_covariance_matrix()

# establishing parameters for optimizer
rf = 0.5  
budget = num_assets // 2   

In [ ]:
# establishing a PO problem instance & converting to a quadratic program

portfolio = PortfolioOptimization(
    expected_returns=ER, covariances=cov, risk_factor=rf, budget=budget
)
qp = portfolio.to_quadratic_program()
qp

In [ ]:
# printing optimization results NumPyMinimumEigensolver (classical method)

mes = NumPyMinimumEigensolver()
eigensolver = MinimumEigenOptimizer(mes)

result = eigensolver.solve(qp)

print_result(result)

In [ ]:
# organizing print results

def print_result(result):
    selection = result.x
    value = result.fval
    print("Optimal: selection {}, value of obj func {:.4f}".format(selection, value))


In [ ]:
eigenstate = result.min_eigen_solver_result.eigenstate
eigenvector = eigenstate if isinstance(eigenstate, np.ndarray) else eigenstate.to_matrix()
probabilities = np.abs(eigenvector) ** 2
i_sorted = reversed(np.argsort(probabilities))

In [ ]:
print("\n----------------- results ---------------------")
print("selection\tvalue\t\tprobability")
for i in i_sorted:
    x = index_to_selection(i, num_assets)
    value = QuadraticProgramToQubo().convert(qp).objective.evaluate(x)
    probability = probabilities[i]
    print("%10s\t%.4f\t\t%.4f" % (x, value, probability))
        